In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# Parâmetros do modelo
L = 128 # Comprimento da grade
T = 2.5 # Temperatura
J = 1 # Cte arbitrária

beta = 1 / T # Aprox. desconsidera Kb
p = 1 - np.exp(-2 * beta * J) # Probabilidade de ligação entre spins vizinhos no algoritmo de cluster
n_passos = 4000  # Número de passos

# Gera spins aleatórios na matriz LxL e salva na variável Spin
spins = np.random.choice([-1, 1], size=(L, L))

# Função para selecionar os vizinhos próximos 'em cruz/+' (com contorno periódico)
def vizinhos(i,j,L):
    return [((i+1)%L,j), # vizinho à direita
            ((i-1)%L,j), # vizinho à esquerda
            (i,(j+1)%L), # vizinho abaixo
            (i,(j-1)%L)] # vizinho acima

# Algoritmo de Wolff
def wolff_cluster_flip(spins,L,p):
    i, j = np.random.randint(0,L, size=2) # Escolhe um valor [0,128) e seleciona a coordenada na matriz
    cluster = {(i,j)} # Agrupamento dos spins que farão parte do grupo a ser invertido.
    spins_fronteira = [(i,j)] # Lista de spins que serão avaliados
    original_spin = spins[i,j] # Spin que veio na matriz na posição ixj

    while spins_fronteira:
        x, y = spins_fronteira.pop()
        for nx, ny in vizinhos(x, y, L):
            if (nx, ny) not in cluster and spins[nx, ny] == original_spin:
                if np.random.rand() < p:
                    cluster.add((nx, ny))
                    spins_fronteira.append((nx, ny))
    for x, y in cluster:
        spins[x,y] *= -1
    return spins

# Seleção dos frames
frames = []
intervalo_frame = 30
for passo in range(n_passos):
    wolff_cluster_flip(spins,L,p)
    if passo % intervalo_frame == 0:
        spin_frame = np.array(spins)
        frames.append(spin_frame.copy())

# Preparando a figura e animação
fig, ax = plt.subplots(figsize=(5,5))
im = ax.imshow(frames[0], cmap="summer", animated=True)
ax.set_title(f"Evolução de Spins (T = {T}, L={L})")
plt.tight_layout()
plt.axis("off")

# Lista para guardar estados da simulação
def update(frame):
    im.set_array(frames[frame])
    return [im]

# Criando animação
ani = FuncAnimation(fig, update, frames=len(frames), interval=200, blit=True)

# Salvando como GIF
ani.save(f"wolff_ising_simulation_T{T}_L{L}.gif", writer=PillowWriter(fps=5))
plt.show()
print(f"GIF salvo como 'wolff_ising_simulation_T{T}_{L}.gif'")
